<a href="https://colab.research.google.com/github/supertime1/BP_PPG/blob/master/Simplified_BP_Data_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run all cells to generate cleaned data

In [0]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import wfdb
import sklearn
from sklearn import preprocessing
import io
import pickle
import numba
from numba import jit
import tensorflow as tf
from scipy.signal import find_peaks

In [0]:
@jit(nopython=True)
def flat_line(signals,threshold = 0, percent = .15):
  clean_signals = []
  #create a list to store the index of the removed segments, this will be used
  #to remove the PPG signals with same index
  rm_list = []
  for i in range(len(signals)):
    #use np.diff to find consecutive points: diff = [i] - [i+1]
    signal_diff = np.diff(signals[i])
    #change value less than threshold to 0, and the rest to 1
    less = np.abs(signal_diff) <= threshold
    more = np.abs(signal_diff) > threshold
    signal_diff[less] = 0
    signal_diff[more] = 1
    #calculate what percent of 0 in the signal, remove the entire signal if 
    #percentage is higher than defined percent
    zero_per = np.sum(signal_diff==0)/len(signal_diff)
    if zero_per < percent:
      clean_signals.append(signals[i])
    else:
      rm_list.append(i)
    
    #track the progress for impatient programmer
    #if i%10000 == 0:
      #print("Processing on", i, "th sample")

  return clean_signals,rm_list

In [0]:
def generate_segment_data(source,seg_len):
  signals =[]
  for signal in source:
    for i in range(int(len(signal)/seg_len)):
      seg = signal[seg_len*i:seg_len*(i+1)]
      signals.append(seg)
#convert list into a numpy array and change its dim from (num of records, seg_len, 1) to (num of records, seg_len)
  signals = np.asarray(list(map(lambda x: np.reshape(x,7500),signals)))

  return signals

In [0]:
def peak_segmentation(signal,distance = 60):
  valleys, _ = find_peaks(signal*-1, distance=distance)
  
  segments = []
  for i in range(len(valleys)-1):
    seg = signal[valleys[i]:valleys[i+1]]
    segments.append(seg)
  
  return segments

In [0]:
#flat_peak: remove cycles that has flat peak in a 1min signal
#input: cycles: 1 min signals that consisits many heart beat cycles
#       tolerance: number of acceptable flat point 
#output: cleaned 1 min signals by removing flat peak cycles
def flat_peak(cycles, tolerance = 2):
  
  clean_cycles = []

  for i in range(len(cycles)):
    if len(np.argwhere(cycles[i] == np.amax(cycles[i]))) > tolerance:
      continue;
    else: 
      clean_cycles.append(cycles[i])

  return clean_cycles

In [0]:
#flat_peak_remove:remove 1min segements that has flat peak
#input: signals: A list of list (i.e. a list of 1 min signals that contains cyclic segments: ABP_ps_signals)
#       seg_ratio: threhold to remove the whole 1min signal (i.e. if the ratio of 
#                   "No. of cleaned flat peak cycles"/"No. of original 1min cycles" 
#                   is less than this treshold, then the whole 1min signal of cycles will be removed )
#output: cleaned_segments: a list of cleaned 1min signals
#        remova_index: a list of removed 1min signals
def flat_peak_remove(signals, seg_ratio = 0.95, tolerance = 2):

  clean_segments = []
  remove_index = [] 

  for i in range(len(signals)):
    #in case some lists are empty
    if signals[i] == []: 
      remove_index.append(i) 
      continue  

    clean_cycles = flat_peak(signals[i],tolerance)
    
    if len(clean_cycles)/(len(signals[i])) < seg_ratio: 
      remove_index.append(i)
      continue
    
    clean_segments.append(clean_cycles)

  return clean_segments, remove_index

In [0]:
#bp_ground_truth: A function that returns average systolic and diastolic value 
#                  of a 1min data
#Inputs: signals: ABP_fpr_signals
#      tolerance: acceptable fluctuations (max - min) of either systolic and diastolic \
#                 during a 1min measurement
#outputs: gt_ls: a list of ground truth for a list of 1min signals
#         remove_index: a list of 1 min signals that has either systolic or diastolic \
#                       fluctuation > tolerance
def bp_ground_truth(signals, tolerance = 20):
  gt_ls = []
  remove_index = []
  for i in range(len(signals)):
    cycles = signals[i]    #a list of cycles in 1min signal
    
    cyc_sys_list = []
    cyc_dia_list = []
    for j in range(len(cycles)):
      cyc_sys_list.append(max(cycles[j]))
      cyc_dia_list.append(cycles[j][0])

    if np.max(np.asarray(cyc_sys_list)) - np.min(np.asarray(cyc_sys_list)) > tolerance \
    or np.max(np.asarray(cyc_dia_list)) - np.min(np.asarray(cyc_dia_list)) > tolerance:
      remove_index.append(i)
      continue

    else: gt_ls.append([np.average(np.asarray(cyc_sys_list)),
                  np.average(np.asarray(cyc_dia_list))])

  return gt_ls, remove_index

In [0]:
from scipy.signal import butter, lfilter
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [0]:
#use numba to improve the speed of for loop
@jit(nopython=True)
def hampel_filter_forloop_numba(input_series, window_size, n_sigmas=3):
    
    n = len(input_series)
    new_series = input_series.copy()
    k = 1.4826 # scale factor for Gaussian distribution
    #indices = []
    
    for i in range((window_size),(n - window_size)):
        x0 = np.nanmedian(input_series[(i - window_size):(i + window_size)])
        S0 = k * np.nanmedian(np.abs(input_series[(i - window_size):(i + window_size)] - x0))
        if (np.abs(input_series[i] - x0) > n_sigmas * S0):
            new_series[i] = x0
            #indices.append(i)
    
    return new_series#, indices

In [0]:
def process_data(directory):
  #load data
  ABP_names = glob.glob(directory + "ABP*.pkl")
  PPG_names = glob.glob(directory + "PPG*.pkl")
  assert(len(ABP_names) == len(PPG_names))
  for i in range(len(ABP_names)):
    
    print("processing", i, "th data")

    with open(ABP_names[i], "rb") as fp:
      ABP_raw_signals = pickle.load(fp)

    with open(PPG_names[i], "rb") as fp:
      PPG_raw_signals = pickle.load(fp)

    #remove flat lines on record level: ABP: 10%, PPG: 10%
    _, PPG_rm_list = flat_line(PPG_raw_signals,0,percent=0.10)
    _, ABP_rm_list = flat_line(ABP_raw_signals,0,percent=0.10)
    
    ABP_list = pd.DataFrame(ABP_rm_list)
    PPG_list = pd.DataFrame(PPG_rm_list)

    try:
      total_list = ABP_list.merge(PPG_list,how="outer")
    except:
      total_list = pd.concat([ABP_list,PPG_list],axis=0)
      
    removal_list=total_list.values.tolist()
    ABP_cl_signals = np.delete(ABP_raw_signals,total_list,0)
    PPG_cl_signals = np.delete(PPG_raw_signals,total_list,0)  

    #segment into 1min data
    ABP_seg_signals = generate_segment_data(ABP_cl_signals, 7500)
    PPG_seg_signals = generate_segment_data(PPG_cl_signals, 7500)

    #remove flat lines on 1min segment level: ABP: 2%, PPG: 5%
    _, ABP_seg_rm_list = flat_line(ABP_seg_signals,0,percent=0.02)
    _, PPG_seg_rm_list = flat_line(PPG_seg_signals,0,percent=0.05)

    ABP_seg_list = pd.DataFrame(ABP_seg_rm_list)
    PPG_seg_list = pd.DataFrame(PPG_seg_rm_list)

    try:
      total_seg_list = ABP_seg_list.merge(PPG_seg_list,how="outer")
    except:
      total_seg_list = pd.concat([ABP_seg_list,PPG_seg_list],axis=0)
    
    removal_list=total_list.values.tolist()
    ABP_seg_cl_signals = np.delete(ABP_seg_signals,total_seg_list,0)
    PPG_seg_cl_signals = np.delete(PPG_seg_signals,total_seg_list,0)

    #PROCESS ABP SIGNAL
    #1.peak segmentation: distance = 60
    ABP_ps_signals = [peak_segmentation(i, distance = 60) for i in ABP_seg_cl_signals]
    #2.flat peak removal: seg_ratio = 0.95, tolerance = 2
    ABP_fpr_signals, remove_index = flat_peak_remove(ABP_ps_signals,0.95,2)
    #3.remove corresponding PPG signal
    PPG_fpr_signals = np.delete(PPG_seg_cl_signals,remove_index,0)
    #4.generate ground truth ABP: tolerance = 30 mmHg
    gt_ls, gt_rm_list= bp_ground_truth(ABP_fpr_signals, tolerance=20)
    #5.remove corresponding PPG signals
    PPG_gt_signals = np.delete(PPG_fpr_signals,gt_rm_list,0)

    #PROCESS PPG SIGNAL
    #1.standardize PPG signal
    PPG_norm_signals = [sklearn.preprocessing.robust_scale(i) for i in PPG_gt_signals]
    #2.band pass filter on PPG sinal
    PPG_bf_signals =[butter_bandpass_filter(i,0.5,8,125,order=4) for i in PPG_norm_signals]
    #3. hampel filter
    PPG_hf_signals = [hampel_filter_forloop_numba(i, 6) for i in PPG_bf_signals]
    #4. resample signal
    ##PLACEHOLDER for resampling signal to a lower frequency, if needed
    with open(directory + "BP_data" + "_" + str(i), "wb") as fp:
      pickle.dump(PPG_hf_signals,fp)

    with open(directory + "BP_label" + "_" + str(i), "wb") as fp:
      pickle.dump(gt_ls,fp)

  return None

In [0]:
directory = 'D:/WFDB//matched/BP/Original Data/'
process_data(directory)